# 全身ポーズじゃんけん

ポーズ推定を用いて全身で戦うじゃんけんを考えました。やっている人は推定結果を見るまで自分が何の手を出しているのか分かりません。コンピュータ上での知的処理と、このポーズはグーっぽいんじゃないか？みたいな人間の感性を駆使したエキサイティングなじゃんけんです。全力でグーチョキパーを全身で表現することが重要です。

1. 評価用のグーチョキパーがラベル付された数枚の画像を用意します。これは僕がそれっぽいと思うポーズを撮影しました。
2. 次に、じゃんけんを行う写真を用意します。これは複数人の人間が全身写っているものなら何でも良いです。
3. じゃんけんを行う写真に対し、ポーズ推定を行います。
4. 推定ポーズがどの評価ポーズに近いのかNearest Neighborを行います。
5. じゃんけんを行う写真に写っている各人のじゃんけんの手が確定します。
6. 勝敗を判定して表示します。

Let's start!

## Install MMPose

ポーズ推定にはOpenMMLabのMMPoseというライブラリを用います。以下のセルを順番に実行することで環境ができます。

We recommend to use a conda environment to install mmpose and its dependencies. And compilers `nvcc` and `gcc` are required.

In [ ]:
# check NVCC version
!nvcc -V

# check GCC version
!gcc --version

# check python in conda environment
!which python

In [ ]:
# install dependencies: (use cu111 because colab has CUDA 11.1)
%pip install torch==1.10.0+cu111 torchvision==0.11.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
%pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.10.0/index.html

# install mmdet for inference demo
%pip install mmdet

# clone mmpose repo
%rm -rf mmpose
!git clone https://github.com/open-mmlab/mmpose.git
%cd mmpose

# install mmpose dependencies
%pip install -r requirements.txt

# install mmpose in develop mode
%pip install -e .

In [ ]:
# Check Pytorch installation
import torch, torchvision

print('torch version:', torch.__version__, torch.cuda.is_available())
print('torchvision version:', torchvision.__version__)

# Check MMPose installation
import mmpose

print('mmpose version:', mmpose.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version

print('cuda version:', get_compiling_cuda_version())
print('compiler information:', get_compiler_version())

## ポーズ推定 → じゃんけん

ポーズ推定にはObject Detectionを先に行い、推定されたバウンディングボックス内でkeypoints推定を行うトップダウン式と、先にkeypoints推定を行って、その後各keypointsがどの人物に属するのかを推定するボトムアップ式があります。今回は、各人物の区別が重要になるので、トップダウン式を採用します。

データセットは COCO dataset (https://cocodataset.org/#home) を用います。
ネットワークは、Object Detectionには HRNet、Pose Estimationには Faster RCNNを用います。

じゃんけんの手の判定にはNearest Neighborを用います。一般にポーズ推定の評価にはObject Keypoints Similarity (OKS) を用いますが、OKSは評価ポーズと離れているポーズに対して0に限りなく近い値を出力するため、今回の用途には適しません。よって、シンプルにNearest Neighborで推定ポーズに最も近い評価ポーズのラベル (手) を出力します。

In [ ]:
import cv2, glob
import numpy as np
from mmpose.apis import (inference_top_down_pose_model, init_pose_model, get_track_id,
                         vis_pose_result, process_mmdet_results)
from mmdet.apis import inference_detector, init_detector

local_runtime = True

pose_config = 'configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/coco/hrnet_w48_coco_256x192.py'
pose_checkpoint = 'https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth'
det_config = 'demo/mmdetection_cfg/faster_rcnn_r50_fpn_coco.py'
det_checkpoint = 'https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'

# initialize pose model
pose_model = init_pose_model(pose_config, pose_checkpoint)
# initialize detector
det_model = init_detector(det_config, det_checkpoint)

my_imgs = glob.glob('mypose/*.JPG')

my_pose_results = []

for my_img in my_imgs:
    # inference detection
    mmdet_results = inference_detector(det_model, my_img)

    # detectionの結果から人間を抽出
    person_results = process_mmdet_results(mmdet_results, cat_id=1)

    # inference pose
    pose_results, returned_outputs = inference_top_down_pose_model(
        pose_model,
        my_img,
        person_results,
        bbox_thr=0.3,
        format='xyxy',
        dataset=pose_model.cfg.data.test.type)

    pose_results, next_id = get_track_id(pose_results, None, 0)

    my_pose_results.append(pose_results[0])

    # show pose estimation results
    vis_result = vis_pose_result(
        pose_model,
        my_img,
        pose_results,
        dataset=pose_model.cfg.data.test.type,
        show=False)
    # reduce image size
    vis_result = cv2.resize(vis_result, dsize=None, fx=0.5, fy=0.5)


# print(len(my_pose_results) ,my_pose_results)

# 評価poseの正規化
for pose in my_pose_results:
    bbox = pose['bbox']
    keypoints = pose['keypoints']
    area =pose['area']
    keypoints[:, 0] -= bbox[0]
    keypoints[:, 1] -= bbox[1]
    keypoints /= area

# 評価ポーズに手を対応させる
my_pose_results[0].update(hand='scissors')
my_pose_results[1].update(hand='paper')
my_pose_results[2].update(hand='rock')
my_pose_results[3].update(hand='scissors')
my_pose_results[4].update(hand='scissors')
my_pose_results[5].update(hand='paper')
my_pose_results[6].update(hand='rock')

hands = ['scissors', 'paper', 'rock', 'scissors', 'scissors', 'paper', 'rock']

# ポーズ推定してじゃんけんを行う画像
img = 'IMG_0001.JPG'

# inference detection
mmdet_results = inference_detector(det_model, img)

# detectionの結果から人間を抽出
person_results = process_mmdet_results(mmdet_results, cat_id=1)

# inference pose
pose_results, returned_outputs = inference_top_down_pose_model(
    pose_model,
    img,
    person_results,
    bbox_thr=0.8,
    format='xyxy',
    dataset=pose_model.cfg.data.test.type)

# Player番号を設定
pose_results, next_id = get_track_id(pose_results, None, 0)

# print(pose_results)

# Nearest Neighbor
def janken(pose, my_pose_results):
    hand_list = []
    keypoints = pose['keypoints'][:,:2]
    # print(keypoints)
    for hand_pose in my_pose_results:
        hand_keypoints = hand_pose['keypoints'][:,:2]
        dist = np.sqrt(np.sum(np.square(keypoints-hand_keypoints)))
        hand_list.append(dist)
    hand = [hands[hand_list.index(min(hand_list))],pose['track_id'], 'Lose']
    return hand

# display result
img = cv2.imread(img)
janken_result = []
for i in range(len(pose_results)):
    janken_result.append(janken(pose_results[i], my_pose_results))
    current_player = str(janken_result[i][1])
    cv2.putText(img, 'P'+ current_player + ': ' + janken_result[i][0], (int(pose_results[i]['bbox'][0]),int(pose_results[i]['bbox'][1])), cv2.FONT_HERSHEY_PLAIN, 4, (0, 0, 255), 5, cv2.LINE_AA)
hand_pool = set()
for hand in janken_result:
    hand_pool.add(hand[0])
if len(hand_pool) == 3 or len(hand_pool) == 1:
    for hand in janken_result:
        hand[2] = 'Draw'
elif 'scissors' not in hand_pool:
    for hand in janken_result:
        if hand[0] == 'paper':
            hand[2] = 'Win!'
elif 'rock' not in hand_pool:
    for hand in janken_result:
        if hand[0] == 'scissors':
            hand[2] = 'Win!'
elif 'paper' not in hand_pool:
    for hand in janken_result:
        if hand[0] == 'rock':
            hand[2] = 'Win!'  
print('janken result', janken_result)
for i in range(len(janken_result)):
    cv2.putText(img, janken_result[i][2], (int(pose_results[i]['bbox'][0]),int(pose_results[i]['bbox'][1])+50), cv2.FONT_HERSHEY_PLAIN, 4, (0, 255, 0), 5, cv2.LINE_AA)

# show pose estimation results
vis_result = vis_pose_result(
    pose_model,
    img,
    pose_results,
    dataset=pose_model.cfg.data.test.type,
    show=False)
# reduce image size
vis_result = cv2.resize(vis_result, dsize=None, fx=0.5, fy=0.5)

if local_runtime:
    from IPython.display import Image, display
    import tempfile
    import os.path as osp
    with tempfile.TemporaryDirectory() as tmpdir:
        file_name = osp.join(tmpdir, 'pose_results.png')
        cv2.imwrite(file_name, vis_result)
        display(Image(file_name))
else:
    cv2_imshow(vis_result)

## 補足

評価ポーズは自分で撮影して```/mypose/```においてください。面白いポーズを撮りましょう。これらに対してNearest Neighborを用いています。

In [ ]:
import glob
import numpy as np
from PIL import Image
import matplotlib.pyplot  as plt

if __name__ == "__main__":
    imgs = glob.glob('mypose/*.JPG')
    image00 = Image.open(imgs[0])
    image01 = Image.open(imgs[1])
    image02 = Image.open(imgs[2])
    image03 = Image.open(imgs[3])
    image04 = Image.open(imgs[4])
    image05 = Image.open(imgs[5])
    image06 = Image.open(imgs[6])
 
    images = [image00, image01, image02, image03, image04, image05, image06]
    hands = ['scissors', 'paper', 'rock', 'scissors', 'scissors', 'paper', 'rock']
 
    fig = plt.figure(facecolor="white", figsize=(12,8))
    for i, im in enumerate(images):
        fig.add_subplot(3,3,i+1).set_title(hands[i])
        plt.xticks([])
        plt.yticks([])
        plt.imshow(im)
 
    plt.show()